In [63]:
import os
import glob
import numpy as np

In [64]:
data_path = './data'
train_data_path = os.path.join(data_path, 'Train')
java_folder_path = train_data_path + '/*.java'
        
file_pair_list = []

for _, folder_name in enumerate(glob.glob(train_data_path + '/*/')):
    file_pair = []
    for _, file_name in enumerate(glob.glob(folder_name + '/*.java')):
        file_pair.append(file_name)
    
    file_pair_list.append(file_pair)

with open("./data/Train/00af3420_5449d33c/00af3420.java", 'r') as file1:
    file1_string = file1.read().replace('\n', '')
    print(file1_string)

import java.util.*;import java.io.*;public class Main {    public static void main(String[] args) throws IOException {        Scanner sc = new Scanner(System.in);        PrintWriter pw = new PrintWriter(System.out);        int t = sc.nextInt();        while(t-- > 0){            int n = sc.nextInt();            String[] s = new String[n];            for(int i=0; i<n; i++)                s[i] = sc.next();            int MAX = 0;            for(char c = 'a'; c <= 'e'; c++){                PriorityQueue<Integer> pq = new PriorityQueue<>(Collections.reverseOrder()); //Big comes in top;                for(int i=0; i<n; ++i) {                    int curChar = 0;                    int otherChar = 0;                    for(int j=0; j<s[i].length(); j++) {                        if(s[i].charAt(j) == c)                            curChar++;                        else                            otherChar++;                    }                    int diff = curChar - otherChar;                  

In [65]:
def read_tokens(text, bag, bag_aux):
    
    text = text + " "
    word_amount = {}
    words_full = bag
    word = ""

    for letter in text:
        if word not in word_amount:
            word_amount[word] = 1
        else: 
            word_amount[word] += 1

        if letter not in bag_aux:
            word += letter.lower()
        elif word != '':
            words_full.append(word)
            word = ''        
        else:
            word = ""
    return words_full, word_amount

In [66]:
def getNgrams(n, text,allNgrams):
    nGrams = {}
    for i in range(len(text)-n+1):
        current = text[i]
        for j in range(i+1,n+i):
            current += " " + (text[j])
        if current not in allNgrams:
            allNgrams.append(current)
        if current not in nGrams:
            nGrams[current] = 1
        else:
            nGrams[current] += 1
    return nGrams,allNgrams


In [67]:
def getVector(word_list, full_word_list):
    vector = []
    for word in full_word_list:
        if word not in word_list:
            vector.append(0)
        else:
            vector.append(word_list[word])
    return vector

In [68]:
%run "Lexer.py"
import sys
from Lexer import *

def getTokens(filepath):

    token_words = []

    lexer = Lexer(filepath)
    token = lexer.scan()
    while token.getTag() != Tag.EOF:
        token_words.append(str(token))
        token = lexer.scan()
    token_words.append("END")

    return token_words


In [69]:
allNgrams = []

token_words1 = getTokens("data/Train/00af3420_5449d33c/5449d33c.java") 
token_words2 = getTokens("data/Train/00af3420_5449d33c/00af3420.java")
nGrams1,allNgrams = getNgrams(3,token_words1,allNgrams)
nGrams2,allNgrams = getNgrams(3,token_words2,allNgrams)

vector1 = getVector(nGrams1,allNgrams)
vector2 = getVector(nGrams2,allNgrams)

print("1: ")
print(nGrams1)
print("2: ") 
print(nGrams2)
print("ALL: ")
print(allNgrams)

print("vector1:")
print(vector1)
print("vector2:")
print(vector2)

1: 
{"'IMPORT' 'JAVA' '.'": 3, "'JAVA' '.' 'IO'": 1, "'.' 'IO' '.'": 1, "'IO' '.' '*'": 1, "'.' '*' ';'": 3, "'*' ';' 'IMPORT'": 2, "';' 'IMPORT' 'JAVA'": 2, "'JAVA' '.' 'UTIL'": 1, "'.' 'UTIL' '.'": 1, "'UTIL' '.' '*'": 1, "'JAVA' '.' 'LANG'": 1, "'.' 'LANG' '.'": 1, "'LANG' '.' '*'": 1, "'*' ';' 'PUBLIC'": 1, "';' 'PUBLIC' 'CLASS'": 1, "'PUBLIC' 'CLASS' 'C'": 1, "'CLASS' 'C' '{'": 1, "'C' '{' 'PUBLIC'": 1, "'{' 'PUBLIC' 'STATIC'": 1, "'PUBLIC' 'STATIC' 'VOID'": 1, "'STATIC' 'VOID' 'MAIN'": 1, "'VOID' 'MAIN' '('": 2, "'MAIN' '(' 'STRING'": 1, "'(' 'STRING' '['": 1, "'STRING' '[' ']'": 2, "'[' ']' 'ARGS'": 1, "']' 'ARGS' ')'": 1, "'ARGS' ')' '{'": 1, "')' '{' 'INPUTSTREAM'": 1, "'{' 'INPUTSTREAM' 'INPUTSTREAM'": 1, "'INPUTSTREAM' 'INPUTSTREAM' '='": 1, "'INPUTSTREAM' '=' 'SYSTEM'": 1, "'=' 'SYSTEM' '.'": 2, "'SYSTEM' '.' 'IN'": 1, "'.' 'IN' ';'": 1, "'IN' ';' 'OUTPUTSTREAM'": 1, "';' 'OUTPUTSTREAM' 'OUTPUTSTREAM'": 1, "'OUTPUTSTREAM' 'OUTPUTSTREAM' '='": 1, "'OUTPUTSTREAM' '=' 'SYSTEM'

In [70]:
def cosineSimilarity(word_vectors):
    
    for i in range(len(word_vectors)):
        word_vectors[i] = np.array(word_vectors[i])
        
    numerator = word_vectors[0]

    for i in range(1, len(word_vectors)):
        numerator = numerator * word_vectors[i]
    
    numerator = np.sum(numerator)

    denominator = 1

    for i in range(len(word_vectors)):
        denominator *= np.sqrt(np.sum(word_vectors[i] ** 2))

    return numerator / denominator

In [71]:
 print('Cosine similarity:', cosineSimilarity([vector1, vector2]))

Cosine similarity: 0.6349800620156187
